In [ ]:
# Got Process
# !git clone https://github.com/Ucas-HaoranWei/GOT-OCR2.0.git 
# !pip install gdown

# !pip install -e . Go to /home/ubuntu/OCR_Test/GOT-OCR2.0/GOT-OCR-2.0-master/pyproject.toml
# !pip install ninja
# !pip install flash-attn --no-build-isolation
# Get The model
# import gdown
# file_id = '1OQrXq_NB_QbJD9yPab6MSj0mcUD4DcrX'
# url = f'https://drive.google.com/uc?id={file_id}'
# output = 'GOT_weights.zip'
# gdown.download(url, output, quiet=False)
# !unzip /home/ubuntu/OCR_Test/GOT_weights.zip -d /home/ubuntu/OCR_Test
# Paste GoT.py to /home/ubuntu/OCR_Test/GOT-OCR2.0/GOT-OCR-2.0-master/GOT/demo

In [10]:
import os
import re
import pandas as pd
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import easyocr

# Create an OCR reader for English
reader = easyocr.Reader(['en'])

# EASY OCR
# Function to check if the text matches the specific pattern
def matches_pattern(text):
    # Regex to match a dash at the fourth last character and last three are digits
    pattern = r'.*-\d{3}$'
    return re.match(pattern, text)

# Function to process a single image file
def process_single_image_easyOCR(image_path):
    # Check if the file is an image
    if not image_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')):
        return "Invalid image file format"
    
    # Perform OCR on the image
    result = reader.readtext(image_path)
    
    # Extract text from each result tuple
    extracted_texts = [text[1] for text in result]
    # print(extracted_texts)
    # Filter texts that match the pattern
    filtered_texts = [text for text in extracted_texts if matches_pattern(text)]
    
    # Return results
    return filtered_texts if filtered_texts else ["No match"]


#DOC OCR

def extract_text_from_json(doc: dict) -> str:
    text_values = []
    for page in doc.get('pages', []):
        for block in page.get('blocks', []):
            for line in block.get('lines', []):
                words = line.get('words', [])
                line_text = ' '.join(word['value'] for word in words if 'value' in word)
                text_values.append(line_text)
    return '\n'.join(text_values)
    
def extract_sku(text: str) -> list:
    # Regular expression to match the SKU pattern (8 to 12 characters, may include a hyphen)
    sku_pattern = re.compile(r'\b[A-Z0-9]{5,6}-[A-Z0-9]{3,6}\b')
    matches = sku_pattern.findall(text)
    valid_skus = [match for match in matches if 8 <= len(match) <= 12]
    return valid_skus


def process_single_image_DoCTR(image_path: str):
    # Instantiate a pretrained model
    predictor = ocr_predictor(pretrained=True)

    # Process the single image
    doc = DocumentFile.from_images(image_path)
    result = predictor(doc).export()
    text = extract_text_from_json(result)
    # print(text)
    skus = extract_sku(text)
    return skus

import sys
sys.path.append("/home/ubuntu/OCR_Test/GOT-OCR2.0/GOT-OCR-2.0-master/GOT/demo")

from GoT import run_ocr
import re

def GoT(
    image_path: str, 
    model_name: str = "/home/ubuntu/OCR_Test/GOT_weights"
):
    # 1) Perform OCR using your run_ocr function (must already be imported or defined in scope).
    ocr_result = run_ocr(
        image_file=image_path,
        model_name=model_name,
        ocr_type="ocr"
    )
    # print(ocr_result)
    # 2) Extract the substring after the word "assistant".
    pattern_after_assistant = r'assistant\s*(.*)'
    match_assistant = re.search(pattern_after_assistant, ocr_result, flags=re.DOTALL)
    if not match_assistant:
        return []

    substring_after = match_assistant.group(1)

    # 3) Use regex to find 6 alphanumeric chars, a dash, and 3 alphanumeric chars.
    code_pattern = r'\b[a-zA-Z0-9]{6}-[a-zA-Z0-9]{3}\b'
    matches = re.findall(code_pattern, substring_after)

    return matches

def Sku(image_path):
    # Process the image using DoCTR
    result = GoT(image_path)
    # Check if the result is empty
    if not result:  # If result is empty
        # print('No match with GoT')
        result = process_single_image_DoCTR(image_path)
    if not result:
        # print('No match with Doctr')
        result = process_single_image_easyOCR(image_path)
    
    # Return the final result
    return result
# GOT OCR


In [16]:
Output=Sku('/home/ubuntu/OCR_Test/cluster_5_tags/85723434812_4.jpg')
print(Output)

['DC0774-101']
